In [1]:
from langchain_groq import ChatGroq

In [3]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)


In [11]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-43838?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Lead Software Engineer, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologyLead Software Engineer, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc.

In [12]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [13]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Lead Software Engineer, ITC',
 'experience': 'Minimum 7-9 years of full stack working experience in designing and building production grade Applications/Platforms.',
 'skills': ['Strong understanding of front-end and back-end development',
  'Experience on Microservices implementation end to end',
  'Proficient in at least one programming language (Java or Golang or Python or NodeJS)',
  'Strong knowledge of containerization with Kubernetes Deployments and Scaling',
  'Must have Experience in AWS web Services (like EC2, RDS, Lambda, S3, Load Balancer, ECS, EKS etc.)',
  'Experience Building end to end CI/CD pipeline to build, test and deploy to different AWS environments such as lambda, EC2,ECS , EKS etc.',
  'Familiarity with software engineering best practices – including unit tests, code review, version control, production monitoring, etc.',
  'Strong understanding of OOP Concepts, Data Structures and Design Patterns',
  'Strong understanding of Web Services, Caching, Scal

In [20]:
import pandas as pd
loc=r"C:\Users\Chandramouli\Desktop\cold_email_generator\app\sources\my_portfolio.csv"
df = pd.read_csv(loc)
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [23]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [25]:
job = json_res
job['skills']

['Strong understanding of front-end and back-end development',
 'Experience on Microservices implementation end to end',
 'Proficient in at least one programming language (Java or Golang or Python or NodeJS)',
 'Strong knowledge of containerization with Kubernetes Deployments and Scaling',
 'Must have Experience in AWS web Services (like EC2, RDS, Lambda, S3, Load Balancer, ECS, EKS etc.)',
 'Experience Building end to end CI/CD pipeline to build, test and deploy to different AWS environments such as lambda, EC2,ECS , EKS etc.',
 'Familiarity with software engineering best practices – including unit tests, code review, version control, production monitoring, etc.',
 'Strong understanding of OOP Concepts, Data Structures and Design Patterns',
 'Strong understanding of Web Services, Caching, Scalability, REST Principles',
 'Strong understanding of database design and development, including relational databases and NoSQL database',
 'Good Knowledge of computer science fundamentals',
 'Kee

In [26]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}

In [27]:
job


{'role': 'Lead Software Engineer, ITC',
 'experience': 'Minimum 7-9 years of full stack working experience in designing and building production grade Applications/Platforms.',
 'skills': ['Strong understanding of front-end and back-end development',
  'Experience on Microservices implementation end to end',
  'Proficient in at least one programming language (Java or Golang or Python or NodeJS)',
  'Strong knowledge of containerization with Kubernetes Deployments and Scaling',
  'Must have Experience in AWS web Services (like EC2, RDS, Lambda, S3, Load Balancer, ECS, EKS etc.)',
  'Experience Building end to end CI/CD pipeline to build, test and deploy to different AWS environments such as lambda, EC2,ECS , EKS etc.',
  'Familiarity with software engineering best practices – including unit tests, code review, version control, production monitoring, etc.',
  'Strong understanding of OOP Concepts, Data Structures and Design Patterns',
  'Strong understanding of Web Services, Caching, Scal

In [31]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at Attackku.pvt.ltd. Attackku is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Attackku's portfolio: {link_list}
        Remember you are Mohan, BDE at Attackku.pvt.ltd. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Lead Software Engineer for Your Tech Modernization Needs

Dear Hiring Manager,

I came across your job posting for a Lead Software Engineer specializing in Tech Modernization activities, and I'm excited to introduce you to Attackku.pvt.ltd, an AI & Software Consulting company dedicated to facilitating the seamless integration of business processes through automated tools.

Our team of experts has a proven track record of developing scalable, cloud-native applications and platforms that support growing business needs. With a strong understanding of front-end and back-end development, we have experience in Microservices implementation, containerization with Kubernetes Deployments and Scaling, and AWS web Services.

We've successfully built end-to-end CI/CD pipelines to build, test, and deploy to different AWS environments, and our team is well-versed in software engineering best practices, including unit tests, code review, version control, and production monitoring.

Our